In [1]:
# Instalando e atualizando as bibliotecas necessárias
!pip install pyDOE
!pip install -U seaborn

In [2]:
# Importando RASD Toolbox
from RASD_TOOLBOX import *
# Importando Numpy toolbox
import numpy as np
from VIGA_METALICA_VERIFICA import *
from FINITO_FEM_TOOLBOX import *

In [45]:
 
ELEMENTO_ANALISADO = 2

P1 = ['NORMAL', 200E9, 20E9] #E MPa                ?kn/cm2?
P2 = ['NORMAL', 200E6, 20E6] #Fy MPa                  ?kn/cm2?
P3 = ['NORMAL', 10000, 1000] #Fy kn
POP = [10]


#carga G, carga Q, tensao escoamento (Fy) e modulo de elasticidade (ES)
#carta_total = carga G + carga Q

#POP = np.arange(10,1000000, 10000).tolist() # (x,y,Z); x=pop inicial; y=pop final; z=variacao pop

SETUP = {'N_REP': len(POP),
        'POP': POP,
        'N_G': 3,
        'D': 3,
        'VARS': [P1, P2, P3],
        'MODEL': 'MCS'}

In [42]:
def OBJ(X):
    
    E = X[0]
    FY = X[1]
    F1 = X[2]
   
    VIGA = {'H_W': 1 , #mm 
        'T_W': 1, #mm
        'B_F': 1, #mm
        'T_F': 1, #mm
        'W_C': 1, #mm
        'W_T': 1, #mm
        'PARAMETRO_PERFIL': 'DUPLA SIMETRIA', #"DUPLA SIMETRIA, MONO SIMETRIA "
        'TIPO_PERFIL': 'LAMINADO', #"SOLDADO, LAMINADO "
        'GAMMA_A1': 1.1,
        'Z': 1, #cm3
        'E_S': E, #kn/cm2
        'F_Y': FY} #kn/cm2
    
    FRAME_00 = {
            "TYPE_ELEMENT": 0,
            "TYPE_SOLUTION": 0,
            "N_NODES": 8,
            "N_MATERIALS": 1,
            "N_SECTIONS": 1,
            "N_ELEMENTS": 9,
            "N_DOFPRESCRIPTIONS": 6,
            "N_DOFLOADED": 4,
            "N_DOFSPRINGS": 0,
            "COORDINATES": #X,Y
            np.array([
            [0.0,0.0],
            [5.25,0.0],
            [0.0,3.0],
            [5.25,3.0],
            [0.0,6.0],
            [5.25,6.0],
            [0.0,9.0],
            [5.25,9.0]]),

            "ELEMENTS": #NODE 1,NODE 2,MATERIAL ID,SECTION ID,HINGE ID NODE 1,HINGE ID NODE 2
            np.array([
            [0,2,0,0,0,0],
            [1,3,0,0,0,0],
            [2,3,0,0,0,0],
            [2,4,0,0,0,0],
            [3,5,0,0,0,0],
            [4,5,0,0,0,0],
            [4,6,0,0,0,0],
            [5,7,0,0,0,0],
            [6,7,0,0,0,0]]),

            "MATERIALS": #YOUNG,POISSON,DENSITY,THERMAL COEFFICIENT
            np.array([[E, 0.30,1,1E-8]]),                                #ES(young) deve vir do perfil, ver unidades
            "SECTIONS": #AREA,INERTIA 1,INERTIA 2,X GC,Y GC
            np.array([[0.02,0.000067,0.000067,0.05,0.1]]),                   #inercia deve vir do perfil, ver unidades
            "PRESCRIBED DISPLACEMENTS": #NODE,DIRECTION(X=0,Y=1,Z=2),VALUE
            np.array([[0,0,0],
            [0,1,0],
            [0,2,0],
            [1,0,0],
            [1,1,0],
            [1,2,0]]),
            "ELEMENT LOADS": None,
            "NODAL LOADS":  #NODE,DIRECTION(X=0,Y=1,Z=2),VALUE
            np.array([
            [2,0, F1,0],                                           
            [4,0, F1,0],
            [6,0, F1,0],
            [6,1,-F1,0],
            [7,1,-F1,0]]),
            "SPRINGS": None}
    
    FRAME_00_RESULTS = MEF1D(DICTIONARY = FRAME_00)
    ELEMENT = pd.DataFrame(data = FRAME_00_RESULTS[ELEMENTO_ANALISADO])
    M_SD = max(ELEMENT['M'].max(),abs(ELEMENT['M'].min()))
    V_SD = max(ELEMENT['V'].max(),abs(ELEMENT['V'].min()))
    N_SD = max(ELEMENT['N'].max(),abs(ELEMENT['N'].min()))
    D_SD = min(ELEMENT['UY'].max(),abs(ELEMENT['UX'].min()))
    print(ELEMENT['UY'])
    print(ELEMENT['M'])

    L_MAX = ELEMENT['X'].max()
    
    #escolher vigas analisadas e determinar esforcos maximos(M,V,DELTA)
    
    ESFORCOS = {'M_SD': M_SD,
                'V_SD': V_SD,
                'D_SD': D_SD,
                'L_MAX': L_MAX}
    
    R_0, S_0, G_0 = VERIFICACAO_VIGA_METALICA_MOMENTO_FLETOR(VIGA, ESFORCOS) 

    R_1, S_1, G_1 = VERIFICACAO_VIGA_METALICA_ESFORCO_CORTANTE(VIGA, ESFORCOS)
    
    R_2, S_2, G_2 = VERIFICACAO_VIGA_METALICA_DEFORMACAO(VIGA, ESFORCOS)
    
    R = [R_0, R_1, R_2] 
    S = [S_0, S_1, S_2] 
    G = [G_0, G_1, G_2]

    return R, S, G

In [46]:
RESULTS_TEST = RASD_STOCHASTIC(SETUP, OBJ)

0        0.000008
1    -1989.000000
2    -1989.000000
3    -1989.000000
4    -1989.000000
5    -1989.000000
6    -1989.000000
7    -1989.000000
8    -1989.000000
9    -1989.000000
10      -0.000020
Name: UY, dtype: float64
0     28516.258293
1     22813.806012
2     17111.353730
3     11408.901449
4      5706.449168
5         3.996886
6     -5698.455395
7    -11400.907676
8    -17103.359958
9    -22805.812239
10   -28508.264520
Name: M, dtype: float64
0        0.000009
1    -1989.000000
2    -1989.000000
3    -1989.000000
4    -1989.000000
5    -1989.000000
6    -1989.000000
7    -1989.000000
8    -1989.000000
9    -1989.000000
10      -0.000024
Name: UY, dtype: float64
0     26984.906251
1     21588.681451
2     16192.456651
3     10796.231850
4      5400.007050
5         3.782250
6     -5392.442551
7    -10788.667351
8    -16184.892152
9    -21581.116952
10   -26977.341752
Name: M, dtype: float64
0        0.000008
1    -1989.000000
2    -1989.000000
3    -1989.000000
4    -1989.00000

In [47]:
RESULTS_TEST[0]['TOTAL RESULTS']

,X_0,X_1,X_2,R_0,R_1,R_2,S_0,S_1,S_2,G_0,G_1,G_2,I_0,I_1,I_2
0,2.533435e+11,2.191055e+08,10335.999712,1.991868e+08,1.0,1.5,28516.258293,10861.813869,0.000008,-1.991583e+08,10860.813869,-1.499992,0.0,1.0,0.0
1,2.013360e+11,1.808458e+08,9780.723168,1.644053e+08,1.0,1.5,26984.906251,10278.523429,0.000009,-1.643783e+08,10277.523429,-1.499991,0.0,1.0,0.0
2,2.400734e+11,2.161105e+08,10592.384886,1.964641e+08,1.0,1.5,29225.370860,11131.914127,0.000008,-1.964349e+08,11130.914127,-1.499992,0.0,1.0,0.0
3,1.722943e+11,1.913099e+08,8801.331297,1.739181e+08,1.0,1.5,24283.656433,9249.620113,0.000010,-1.738938e+08,9248.620113,-1.499990,0.0,1.0,0.0
4,1.976070e+11,1.841562e+08,10470.775024,1.674147e+08,1.0,1.5,28888.749330,11003.695328,0.000010,-1.673859e+08,11002.695328,-1.499990,0.0,1.0,0.0
5,2.268492e+11,2.001864e+08,9626.212954,1.819877e+08,1.0,1.5,26559.990550,10116.673469,0.000008,-1.819611e+08,10115.673469,-1.499992,0.0,1.0,0.0
6,1.958720e+11,1.614258e+08,12030.268729,1.467507e+08,1.0,1.5,33193.090205,12643.214402,0.000012,-1.467175e+08,12642.214402,-1.499988,0.0,1.0,0.0
7,1.922121e+11,2.066508e+08,8166.249231,1.878644e+08,1.0,1.5,22531.568962,8582.251771,0.000008,-1.878418e+08,8581.251771,-1.499992,0.0,1.0,0.0
8,2.216251e+11,2.283266e+08,9523.098579,2.075696e+08,1.0,1.5,26275.793684,10008.423171,0.000008,-2.075434e+08,10007.423171,-1.499992,0.0,1.0,0.0
9,2.089306e+11,1.895274e+08,9575.524161,1.722976e+08,1.0,1.5,26419.271713,10063.073807,0.000009,-1.722712e+08,10062.073807,-1.499991,0.0,1.0,0.0


In [ ]:
DADOS = RESULTS_TEST[0]['TOTAL RESULTS']

In [ ]:
DADOS = RESULTS_TEST[len(POP)-1]['TOTAL RESULTS']

In [ ]:
OPCOES_GRAFICAS = {'NAME': 'WANDER',
                'WIDTH': 0.40, 
                'HEIGHT': 0.20, 
                'X AXIS SIZE': 20,
                'Y AXIS SIZE': 20,
                'AXISES COLOR': '#000000',
                'X AXIS LABEL': '$x_0$ - $P_X (lb)$',
                'LABELS SIZE': 16,
                'LABELS COLOR': '#000000',  
                'CHART COLOR': '#FEB625',
                'KDE': False,
                'DPI': 600, 
                'BINS' : 20,
                'EXTENSION': '.svg'}
# RESULTS X_0 VARIABLE
OPCOES_DADOS = {'DATASET': DADOS, 'COLUMN': 'X_0'}      
# CALL PLOT
RASD_PLOT_1(OPCOES_DADOS, OPCOES_GRAFICAS)

In [ ]:
OPCOES_GRAFICAS = {'NAME': 'WANDER',
                'EXTENSION': '.svg',
                'DPI': 600,
                'WIDTH': 0.300, 
                'HEIGHT': 0.150,              
                'X DATA': 'S_0',
                'Y DATA': 'R_0',
                'X AXIS SIZE': 16,
                'Y AXIS SIZE': 16,
                'AXISES COLOR': '#000000',
                'X AXIS LABEL': '$S_0$',
                'Y AXIS LABEL': '$R_0$',
                'LABELS SIZE': 18,
                'LABELS COLOR': '#000000',
                'LOC LEGEND': 'lower right',
                'TITLE LEGEND': 'Failure index ($I$):'}
# RESULTS
OPCOES_DADOS = {'DATASET': DADOS, 'X DATA': 'S_0', 'Y DATA': 'R_0', 'HUE VALUE': 'I_0'}    
# CALL PLOT
RASD_PLOT_2(OPCOES_DADOS, OPCOES_GRAFICAS)

In [ ]:
OPCOES_GRAFICAS = {'NAME': 'WANDER',
                'EXTENSION': '.svg',
                'DPI': 600,
                'WIDTH': 0.20, 
                'HEIGHT': 0.10,              
                'X DATA': 'S_0',
                'Y DATA': 'R_0',
                'X AXIS SIZE': 20,
                'Y AXIS SIZE': 20,
                'AXISES COLOR': '#000000',
                'X AXIS LABEL': '$S_0$',
                'Y AXIS LABEL': '$R_0$',
                'LABELS SIZE': 16,
                'LABELS COLOR': '#000000',
                'G VALUE': 'G_0',
                'TRANSPARENCY': 0.8,
                'COLOR MAP': 'viridis'}


# RESULTS
OPCOES_DADOS = {'DATASET': DADOS, 'X DATA': 'S_0', 'Y DATA': 'R_0', 'G VALUE': 'G_0'}       

# CALL PLOT
RASD_PLOT_3(OPCOES_DADOS, OPCOES_GRAFICAS)

In [ ]:
plt.hist(RESULTS_TEST[0]['TOTAL RESULTS']['R_0'], bins=50, label='a', alpha=0.5)
plt.hist(RESULTS_TEST[0]['TOTAL RESULTS']['S_0'], bins=50, label='b', alpha=0.5)

In [ ]:
OPCOES_GRAFICAS = {'NAME': 'WANDER',
                    'EXTENSION': '.svg',
                    'DPI': 600,
                    'WIDTH': 0.20,
                    'HEIGHT': 0.10,
                    'X DATA': 'S_0',
                    'Y DATA': 'R_0',
                    'X AXIS SIZE': 20,
                    'Y AXIS SIZE': 20,
                    'AXISES COLOR': '#000000',
                    'X AXIS LABEL': '$G_0 + G_0$',
                    'Y AXIS LABEL': '$SOMATÓRIO$',
                    'LABELS SIZE': 16,
                    'LABELS COLOR': '#000000',
                    'C VALUE': 'G_0',
                    'TRANSPARENCY': 0.8,
                    'COLOR MAP': 'viridis',
                    'BINS': '50',
                    'ALPHA': '0.5'}

# RESULTS
OPCOES_DADOS = {'DATASET': DADOS, 'X DATA': 'S_0', 'Y DATA': 'R_0', 'G VALUE': 'G_0'}       

# CALL PLOT
RASD_PLOT_4(OPCOES_DADOS, OPCOES_GRAFICAS)

In [ ]:
OPCOES_GRAFICAS = {'NAME': 'WANDER',
                    'EXTENSION': '.svg',
                    'DPI': 600,
                    'WIDTH': 0.20,
                    'HEIGHT': 0.10,
                    'X AXIS SIZE': 20,
                    'Y AXIS SIZE': 20,
                    'AXISES COLOR': '#000000',
                    'X AXIS LABEL': 'Número de Simulações (ns)',
                    'Y AXIS LABEL': 'Probabilidade de Falha',
                    'LABELS SIZE': 16,
                    'LABELS COLOR': '#000000',
                    'CHART COLOR': 'black',
                    'POPULATION' : POP,
                    'TYPE' : 'pf'}

# RESULTS
OPCOES_DADOS = {'DATASET': RESULTS_TEST}       

# CALL PLOT
RASD_PLOT_5(OPCOES_DADOS, OPCOES_GRAFICAS)

In [ ]:
OPCOES_GRAFICAS = {'NAME': 'WANDER',
                    'EXTENSION': '.svg',
                    'DPI': 600,
                    'WIDTH': 0.20,
                    'HEIGHT': 0.10,
                    'X AXIS SIZE': 20,
                    'Y AXIS SIZE': 20,
                    'AXISES COLOR': '#000000',
                    'X AXIS LABEL': 'Número de Simulações (ns)',
                    'Y AXIS LABEL': 'Beta',
                    'LABELS SIZE': 16,
                    'LABELS COLOR': '#000000',
                    'CHART COLOR': 'black',
                    'POPULATION' : POP,
                    'TYPE' : 'beta'}

# RESULTS
OPCOES_DADOS = {'DATASET': RESULTS_TEST}       

# CALL PLOT
RASD_PLOT_5(OPCOES_DADOS, OPCOES_GRAFICAS)

In [ ]:
RESULTS_TEST[2]